# Google Firestore (原生模式)

> [Firestore](https://cloud.google.com/firestore) 是一个无服务器的面向文档的数据库，可根据任何需求进行扩展。通过利用 Firestore 的 Langchain 集成，扩展您的数据库应用程序以构建 AI 驱动的体验。

本笔记本将介绍如何使用 [Firestore](https://cloud.google.com/firestore) 来存储向量并使用 `FirestoreVectorStore` 类查询它们。

[![在 Colab 中打开](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googleapis/langchain-google-firestore-python/blob/main/docs/vectorstores.ipynb)

## 开始之前

要运行此Notebook，您需要完成以下操作：

*   [创建 Google Cloud 项目](https://developers.google.com/workspace/guides/create-project)
*   [启用 Firestore API](https://console.cloud.google.com/flows/enableapi?apiid=firestore.googleapis.com)
*   [创建 Firestore 数据库](https://cloud.google.com/firestore/docs/manage-databases)

在 Notebook 的运行时环境中确认访问数据库后，填写以下值并运行单元格，然后再运行示例脚本。

In [ ]:
# @markdown Please specify a source for demo purpose.
COLLECTION_NAME = "test"  # @param {type:"CollectionReference"|"string"}

### 🦜🔗 库安装

集成部分位于独立的 `langchain-google-firestore` 包中，因此我们需要安装它。在本笔记本中，我们还将安装 `langchain-google-genai` 来使用 Google Generative AI 嵌入。

In [ ]:
%pip install -upgrade --quiet langchain-google-firestore langchain-google-vertexai

**仅限 Colab**: 取消注释下面的单元格以重启内核，或使用按钮来重启内核。对于 Vertex AI Workbench，您可以使用顶部的按钮重启终端。

In [ ]:
# # Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

### ☁ 设置您的 Google Cloud 项目
设置您的 Google Cloud 项目，以便在此笔记本中利用 Google Cloud 资源。

如果您不知道项目 ID，可以尝试以下方法：

* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 查看支持页面：[查找项目 ID](https://support.google.com/googleapi/answer/7014113)。

In [ ]:
# @markdown Please fill in the value below with your Google Cloud project ID and then run the cell.

PROJECT_ID = "extensions-testing"  # @param {type:"string"}

# Set the project id
!gcloud config set project {PROJECT_ID}

### 🔐 身份验证

验证您笔记本当前登录的 IAM 用户身份，以便访问您的 Google Cloud 项目。

- 如果您正在使用 Colab 运行此笔记本，请使用下面的单元格并继续。
- 如果您正在使用 Vertex AI Workbench，请参阅此处的设置说明：[https://github.com/GoogleCloudPlatform/generative-ai/tree/main/setup-env](https://github.com/GoogleCloudPlatform/generative-ai/tree/main/setup-env)。

In [ ]:
from google.colab import auth

auth.authenticate_user()

# 基本用法

### 初始化 FirestoreVectorStore

`FirestoreVectorStore` 允许您将新向量存储在 Firestore 数据库中。您可以使用它来存储来自任何模型的嵌入，包括来自 Google Generative AI 的模型。

In [ ]:
from langchain_google_firestore import FirestoreVectorStore
from langchain_google_vertexai import VertexAIEmbeddings

embedding = VertexAIEmbeddings(
    model_name="textembedding-gecko@latest",
    project=PROJECT_ID,
)

# Sample data
ids = ["apple", "banana", "orange"]
fruits_texts = ['{"name": "apple"}', '{"name": "banana"}', '{"name": "orange"}']

# Create a vector store
vector_store = FirestoreVectorStore(
    collection="fruits",
    embedding=embedding,
)

# Add the fruits to the vector store
vector_store.add_texts(fruits_texts, ids=ids)

作为一种简写方式，您可以使用 `from_texts` 和 `from_documents` 方法在单个步骤中初始化并添加向量。

In [ ]:
vector_store = FirestoreVectorStore.from_texts(
    collection="fruits",
    texts=fruits_texts,
    embedding=embedding,
)

In [ ]:
from langchain_core.documents import Document

fruits_docs = [Document(page_content=fruit) for fruit in fruits_texts]

vector_store = FirestoreVectorStore.from_documents(
    collection="fruits",
    documents=fruits_docs,
    embedding=embedding,
)

### 删除向量

您可以使用 `delete` 方法从数据库中删除带有向量的文档。您需要提供要删除的向量的文档 ID。这将从数据库中删除整个文档，包括它可能拥有的任何其他字段。

In [ ]:
vector_store.delete(ids)

### 更新向量

更新向量与添加向量类似。您可以使用 `add` 方法，通过提供文档 ID 和新向量来更新文档的向量。

In [ ]:
fruit_to_update = ['{"name": "apple","price": 12}']
apple_id = "apple"

vector_store.add_texts(fruit_to_update, ids=[apple_id])

## 相似性搜索

您可以使用 `FirestoreVectorStore` 对存储的向量执行相似性搜索。这对于查找相似的文档或文本非常有用。

In [ ]:
vector_store.similarity_search("I like fuji apples", k=3)

In [ ]:
vector_store.max_marginal_relevance_search("fuji", 5)

您可以通过使用 `filters` 参数来添加搜索的预筛选。这对于按特定字段或值进行筛选非常有用。

In [ ]:
from google.cloud.firestore_v1.base_query import FieldFilter

vector_store.max_marginal_relevance_search(
    "fuji", 5, filters=FieldFilter("content", "==", "apple")
)

### 自定义连接与认证

In [ ]:
from google.api_core.client_options import ClientOptions
from google.cloud import firestore
from langchain_google_firestore import FirestoreVectorStore

client_options = ClientOptions()
client = firestore.Client(client_options=client_options)

# Create a vector store
vector_store = FirestoreVectorStore(
    collection="fruits",
    embedding=embedding,
    client=client,
)